<a href="https://colab.research.google.com/github/attililalithadevi/Moodlens-Project/blob/main/Moodlens_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MoodLens:**
- AI-Powered Emotion Detection System

 **Created by**  -  Lalitha Devi Attili

 **Overview**:

 An innovative project leveraging DeepFace and OpenCV to detect
emotions from facial images,featuring real-time webcam capture and file upload options, with results visualized in an interactive bar graph. Built with Python in Google Colab using ipywidgets for a GUI-like experience, demonstrating skills in AI, computer vision, and data visualization.

 **Key Skills Demonstrated:**
 - **AI & Machine Learning:** Utilized pre-trained DeepFace models for emotion recognition
 - **Computer Vision:** Processed images with OpenCV for real-time analysis
 - **Data Visualization**: Designed dynamic bar graphs with Matplotlib
 - **Interactive UI:** Implemented ipywidgets for user-friendly interface in Colab
 - **Problem Solving:** Handled webcam integration and file uploads in a cloud environment

**Install required packages**

In [ ]:
# Install required packages
!pip install deepface opencv-python matplotlib numpy ipywidgets  # Installs essential libraries for AI, image processing, plotting, and widgets

import cv2  # Imports OpenCV for image processing and webcam handling
import matplotlib.pyplot as plt  # Imports Matplotlib for creating visualizations (bar graphs)
from deepface import DeepFace  # Imports DeepFace for emotion detection using pre-trained models
import numpy as np  # Imports NumPy for numerical operations and array handling
from google.colab import files  # Imports Colab's file module for uploading images
from IPython.display import display, Javascript, clear_output  # Imports IPython tools for displaying widgets and JavaScript execution
from google.colab.output import eval_js  # Imports eval_js to run JavaScript in Colab
from base64 import b64decode  # Imports base64 decoding for webcam image data
import ipywidgets as widgets  # Imports ipywidgets for creating interactive UI elements
from io import BytesIO  # Imports BytesIO for handling image data in memory
from PIL import Image  # Imports PIL for image manipulation (not heavily used here but included for potential enhancements)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=56f7c4811fe24f7be03c574f6e576a920967dd97003669dcf8f9ba12cac18759
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
25-04-09 13:38:18 - Directory /root/.deepface has been created
25-04-09 13:38:18 - Directory /root/.deepface/weights has been created


In [12]:

# Function to capture photo from webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Function to analyze emotion
def analyze_emotion(img_path):
    try:
        img = cv2.imread(img_path)
        result = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
        emotions = result[0]['emotion']
        return emotions, img
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None, None

# Function to plot emotions
def plot_emotions(emotions, img):
    emotion_labels = list(emotions.keys())
    emotion_values = list(emotions.values())
    dominant_emotion = max(emotions.items(), key=lambda x: x[1])[0]

    fig = plt.figure(figsize=(12, 5))

    # Display input image
    plt.subplot(1, 2, 1)
    plt.imshow(img[:,:,::-1])
    plt.title("Input Image")
    plt.axis('off')

    # Bar graph
    plt.subplot(1, 2, 2)
    bars = plt.bar(emotion_labels, emotion_values, color='skyblue')
    plt.title('Emotion Analysis Results')
    plt.xlabel('Emotions')
    plt.ylabel('Percentage')
    plt.ylim(0, 100)
    plt.xticks(rotation=45)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
    print(f"This picture is having {dominant_emotion.capitalize()} emotion")

# Button click handlers
def on_capture_clicked(b):
    with output:
        clear_output()
        print("Click 'Capture' in the video feed when ready...")
        try:
            img_path = take_photo()
            emotions, img = analyze_emotion(img_path)
            if emotions and img is not None:
                plot_emotions(emotions, img)
            else:
                print("Failed to analyze the captured image.")
        except Exception as e:
            print(f"Error capturing photo: {e}")

def on_upload_clicked(b):
    with output:
        clear_output()
        uploaded = files.upload()
        if uploaded:
            file_name = list(uploaded.keys())[0]
            with open(file_name, 'wb') as f:
                f.write(uploaded[file_name])
            emotions, img = analyze_emotion(file_name)
            if emotions and img is not None:
                plot_emotions(emotions, img)
            else:
                print("Failed to analyze the uploaded image.")
        else:
            print("No file uploaded!")

# Create widgets
capture_button = widgets.Button(description="Capture from Webcam",
                              button_style='success',
                              tooltip='Click to capture an image from webcam')
upload_button = widgets.Button(description="Upload Image",
                             button_style='info',
                             tooltip='Click to upload an image')
output = widgets.Output()

# Assign button click events
capture_button.on_click(on_capture_clicked)
upload_button.on_click(on_upload_clicked)

# Display the interface
display(widgets.VBox([widgets.Label("Emotion Detection System"),
                     capture_button,
                     upload_button,
                     output]))